# Head

In [ ]:
!pip install mambapy

In [2]:
import torch, transformers, importlib, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [23]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
input_ids = tokenizer("Today is Sunday.", return_tensors="pt")["input_ids"] #.to('cuda')

In [27]:
importlib.reload(InitMamba)
import InitMamba

config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")
config.use_mambapy = True
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config) #.to('cuda')
# model1.train()

res1 = model1(input_ids, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states

In [ ]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(input_ids, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the,\n']
[' is Sunday,\n']


# Check same

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(input_ids).logits - model1(input_ids).logits).abs().sum())